In [1]:
# Import libraries
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Split into training and testing

# Load the labels from the Excel file
labels_path = './Facial Database/labels.csv'
labels_df = pd.read_csv(labels_path)

# Split the data into training and testing sets (80%-20%)
train_df, test_df = train_test_split(labels_df, test_size=0.2, random_state=42)

train_dir = './database/train'
test_dir = './database/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subdirectories for each label
for label in labels_df['label'].unique():
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)

def move_images(dataframe, split_dir):
    for index, row in dataframe.iterrows():
        src_path = os.path.join('./Facial Database', row['pth'])  # Ensure this path is correct
        dst_path = os.path.join(split_dir, row['label'], os.path.basename(row['pth']))  # Use basename to avoid directory duplication
        if os.path.exists(src_path):  # Check if the source file exists
            shutil.copy2(src_path, dst_path)
        else:
            print(f"File {src_path} does not exist.")  # Debugging line to check missing files

# Move images to train and test directories
move_images(train_df, train_dir)
move_images(test_df, test_dir)

print("Files have been distributed into train and test directories.")

Files have been distributed into train and test directories.
